## **Federated Learning for attack classification: 3 nodes sharing gradients**

IDs from this file = **id0xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'dos', data['attack_cat'].replace('dos', 1).replace(['reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace([ 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'generic', data['attack_cat'].replace('generic', 1).replace(['dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'exploits', data['attack_cat'].replace('exploits', 1).replace([ 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(Dense(108, input_shape=input_shape, activation='relu'))
    model.add(layers.Conv2D(filters=16, kernel_size=(1, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(108, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(4)], target_names=['dos', 'reconnaissance', 'generic', 'exploits'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_9140/2733919938.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'ID000.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [9]:
global_model = build_model((24,1,1))

In [10]:
# Filter test and train by not getting normal traffic: label is 1
test_basic = test_basic.loc[test_basic['label'] == 1]
test_plus = test_plus.loc[test_plus['label'] == 1]

In [11]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [12]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [13]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id000.hdf5')

Epoch 1/5
51/51 [==============================] - 3s 61ms/step - loss: 0.5111 - accuracy: 0.9569 - val_loss: 1.4468 - val_accuracy: 0.1444
Epoch 2/5
51/51 [==============================] - 3s 53ms/step - loss: 0.0443 - accuracy: 0.9994 - val_loss: 1.4906 - val_accuracy: 0.1431
Epoch 3/5
51/51 [==============================] - 3s 55ms/step - loss: 0.0122 - accuracy: 0.9995 - val_loss: 1.5498 - val_accuracy: 0.1417
Epoch 4/5
51/51 [==============================] - 3s 56ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 1.6444 - val_accuracy: 0.1414
Epoch 5/5
51/51 [==============================] - 3s 56ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 1.7871 - val_accuracy: 0.1413
Epoch 1/5
51/51 [==============================] - 3s 58ms/step - loss: 4.0405e-04 - accuracy: 1.0000 - val_loss: 2.5839 - val_accuracy: 0.1413
Epoch 2/5
51/51 [==============================] - 3s 54ms/step - loss: 3.8410e-05 - accuracy: 1.0000 - val_loss: 3.3246 - val_accuracy: 0.1413
Epoch 3/5
51

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 98/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 77/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [14]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[2.490201473236084, 0.0], [3.0537493228912354, 0.0], [3.1763179302215576, 0.0]]
Accuracy for iterations:  [[0.8319796919822693, 0.0], [0.8319796919822693, 0.0], [0.8319796919822693, 0.0]]
F1 for iterations:  [[0.7557141574837521, 0.0], [0.7557013693602648, 0.0], [0.7557013693602648, 0.0]]
Precision for iterations:  [[0.6922567069545479, 0.0], [0.6922352459557739, 0.0], [0.6922352459557739, 0.0]]
Recall for iterations:  [[0.8319796639593279, 0.0], [0.8319796639593279, 0.0], [0.8319796639593279, 0.0]]


#### Dataset 3B 

In [15]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')

In [16]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [17]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id001.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [18]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [21]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id001.hdf5')

Epoch 1/5
15/15 [==============================] - 1s 49ms/step - loss: 1.3446 - accuracy: 0.4943 - val_loss: 1.3871 - val_accuracy: 0.1394
Epoch 2/5
15/15 [==============================] - 1s 37ms/step - loss: 0.8474 - accuracy: 0.7260 - val_loss: 1.4083 - val_accuracy: 0.0047
Epoch 3/5
15/15 [==============================] - 1s 38ms/step - loss: 0.5341 - accuracy: 0.9432 - val_loss: 1.4320 - val_accuracy: 0.0000e+00
Epoch 4/5
15/15 [==============================] - 1s 38ms/step - loss: 0.3439 - accuracy: 0.9578 - val_loss: 1.4591 - val_accuracy: 0.0000e+00
Epoch 5/5
15/15 [==============================] - 1s 38ms/step - loss: 0.2521 - accuracy: 0.9593 - val_loss: 1.4868 - val_accuracy: 0.0000e+00
Epoch 1/5
15/15 [==============================] - 1s 57ms/step - loss: 0.2060 - accuracy: 0.9618 - val_loss: 1.5403 - val_accuracy: 0.0000e+00
Epoch 2/5
15/15 [==============================] - 1s 38ms/step - loss: 0.1843 - accuracy: 0.9626 - val_loss: 1.5900 - val_accuracy: 0.0011
Epoc

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 88/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 7.8574e-04


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 90/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 5.7897e-04


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.9997096061706543, 0.0], [0.7288485169410706, 0.0], [0.8146520853042603, 0.0]]
Accuracy for iterations:  [[0.9243753552436829, 0.000785740849096328], [0.9244373440742493, 0.0007030313136056066], [0.9244373440742493, 0.0005789669812656939]]
F1 for iterations:  [[0.9006694012382915, 0.0015702479338842972], [0.9024768401055161, 0.0014050747995702126], [0.9038515695470377, 0.0011572638974994833]]
Precision for iterations:  [[0.8850810222648535, 1.0], [0.9051148713574935, 1.0], [0.9090988690155398, 1.0]]
Recall for iterations:  [[0.9243753487506975, 0.0007857408709317233], [0.9244373488746978, 0.0007030313055704893], [0.9244373488746978, 0.0005789669575286382]]


#### Dataset 3C

In [24]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3C-Part3.csv')

In [25]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id002.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [28]:
global_model = build_model((24,1,1))

In [29]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [30]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [31]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id002.hdf5')

Epoch 1/5
4/4 [==============================] - 0s 81ms/step - loss: 1.2956 - accuracy: 0.3753 - val_loss: 1.3826 - val_accuracy: 0.2868
Epoch 2/5
4/4 [==============================] - 0s 35ms/step - loss: 1.1912 - accuracy: 0.5469 - val_loss: 1.3884 - val_accuracy: 0.2302
Epoch 3/5
4/4 [==============================] - 0s 31ms/step - loss: 1.1017 - accuracy: 0.5998 - val_loss: 1.3942 - val_accuracy: 0.1107
Epoch 4/5
4/4 [==============================] - 0s 33ms/step - loss: 1.0282 - accuracy: 0.6093 - val_loss: 1.3997 - val_accuracy: 0.0516
Epoch 5/5
4/4 [==============================] - 0s 32ms/step - loss: 0.9689 - accuracy: 0.6181 - val_loss: 1.4051 - val_accuracy: 0.0465
Epoch 1/5
4/4 [==============================] - 0s 72ms/step - loss: 0.9100 - accuracy: 0.6423 - val_loss: 1.4118 - val_accuracy: 0.0377
Epoch 2/5
4/4 [==============================] - 0s 34ms/step - loss: 0.8677 - accuracy: 0.6470 - val_loss: 1.4173 - val_accuracy: 0.0340
Epoch 3/5
4/4 [===================

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6268


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 0s 66ms/step - loss: 0.7631 - accuracy: 0.6524 - val_loss: 1.4314 - val_accuracy: 0.0025
Epoch 2/5
4/4 [==============================] - 0s 32ms/step - loss: 0.7555 - accuracy: 0.6615 - val_loss: 1.4288 - val_accuracy: 0.0025
Epoch 3/5
4/4 [==============================] - 0s 34ms/step - loss: 0.7503 - accuracy: 0.6596 - val_loss: 1.4276 - val_accuracy: 0.0038
Epoch 4/5
4/4 [==============================] - 0s 32ms/step - loss: 0.7462 - accuracy: 0.6618 - val_loss: 1.4269 - val_accuracy: 0.0038
Epoch 5/5
4/4 [==============================] - 0s 32ms/step - loss: 0.7416 - accuracy: 0.6637 - val_loss: 1.4260 - val_accuracy: 0.0038
Epoch 1/5
4/4 [==============================] - 0s 72ms/step - loss: 0.7395 - accuracy: 0.6628 - val_loss: 1.4250 - val_accuracy: 0.0075
Epoch 2/5
4/4 [==============================] - 0s 32ms/step - loss: 0.7345 - accuracy: 0.6650 - val_loss: 1.4256 - val_accuracy: 0.0113
Epoch 3/5
4/4 [===================

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 0s 75ms/step - loss: 0.7160 - accuracy: 0.6628 - val_loss: 1.4158 - val_accuracy: 0.0138
Epoch 2/5
4/4 [==============================] - 0s 39ms/step - loss: 0.7100 - accuracy: 0.6694 - val_loss: 1.4116 - val_accuracy: 0.0164
Epoch 3/5
4/4 [==============================] - 0s 44ms/step - loss: 0.7075 - accuracy: 0.6735 - val_loss: 1.4105 - val_accuracy: 0.0201
Epoch 4/5
4/4 [==============================] - 0s 39ms/step - loss: 0.7044 - accuracy: 0.6795 - val_loss: 1.4091 - val_accuracy: 0.0239
Epoch 5/5
4/4 [==============================] - 0s 43ms/step - loss: 0.7008 - accuracy: 0.6725 - val_loss: 1.4067 - val_accuracy: 0.0302
Epoch 1/5
4/4 [==============================] - 0s 69ms/step - loss: 0.7027 - accuracy: 0.6700 - val_loss: 1.4046 - val_accuracy: 0.0252
Epoch 2/5
4/4 [==============================] - 0s 33ms/step - loss: 0.6979 - accuracy: 0.6707 - val_loss: 1.4084 - val_accuracy: 0.0226
Epoch 3/5
4/4 [===================

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.4639


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.351609230041504, 0.0], [1.344989538192749, 0.0], [1.3233356475830078, 0.0]]
Accuracy for iterations:  [[0.7739320397377014, 0.6267730593681335], [0.5587140917778015, 0.5948472023010254], [0.5665261149406433, 0.46391794085502625]]
F1 for iterations:  [[0.8079833909530446, 0.7705722347916719], [0.6807306688355753, 0.7459613639310255], [0.6849748581510946, 0.6338032147800785]]
Precision for iterations:  [[0.8705563500231841, 1.0], [0.9017045766615176, 1.0], [0.9035348578338046, 1.0]]
Recall for iterations:  [[0.7739320478640958, 0.6267730863074314], [0.5587141174282348, 0.5948471940779951], [0.5665261330522661, 0.46391795211116166]]


#### Dataset 3D

In [33]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3D-Part3.csv')

In [34]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [35]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id003.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [36]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [37]:
global_model = build_model((24,1,1))

In [38]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [39]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [40]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id003.hdf5')

Epoch 1/5
34/34 [==============================] - 2s 55ms/step - loss: 1.1687 - accuracy: 0.5011 - val_loss: 1.3323 - val_accuracy: 0.8121
Epoch 2/5
34/34 [==============================] - 2s 53ms/step - loss: 0.3827 - accuracy: 0.9562 - val_loss: 1.1736 - val_accuracy: 0.9773
Epoch 3/5
34/34 [==============================] - 2s 51ms/step - loss: 0.1556 - accuracy: 0.9827 - val_loss: 1.0347 - val_accuracy: 0.9798
Epoch 4/5
34/34 [==============================] - 2s 52ms/step - loss: 0.1061 - accuracy: 0.9831 - val_loss: 0.9319 - val_accuracy: 0.9802
Epoch 5/5
34/34 [==============================] - 2s 51ms/step - loss: 0.0930 - accuracy: 0.9832 - val_loss: 0.8442 - val_accuracy: 0.9809
Epoch 1/5
5/5 [==============================] - 0s 72ms/step - loss: 1.9985 - accuracy: 0.6167 - val_loss: 1.6887 - val_accuracy: 0.0017
Epoch 2/5
5/5 [==============================] - 0s 42ms/step - loss: 1.7584 - accuracy: 0.6167 - val_loss: 1.6267 - val_accuracy: 0.0043
Epoch 3/5
5/5 [=========

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0231


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
34/34 [==============================] - 2s 61ms/step - loss: 0.3565 - accuracy: 0.8715 - val_loss: 0.8832 - val_accuracy: 0.9801
Epoch 2/5
34/34 [==============================] - 2s 56ms/step - loss: 0.1028 - accuracy: 0.9833 - val_loss: 0.6338 - val_accuracy: 0.9814
Epoch 3/5
34/34 [==============================] - 2s 45ms/step - loss: 0.0877 - accuracy: 0.9834 - val_loss: 0.5113 - val_accuracy: 0.9818
Epoch 4/5
34/34 [==============================] - 1s 43ms/step - loss: 0.0819 - accuracy: 0.9837 - val_loss: 0.4113 - val_accuracy: 0.9820
Epoch 5/5
34/34 [==============================] - 1s 43ms/step - loss: 0.0782 - accuracy: 0.9838 - val_loss: 0.3240 - val_accuracy: 0.9827
Epoch 1/5
5/5 [==============================] - 0s 72ms/step - loss: 2.1586 - accuracy: 0.6158 - val_loss: 2.6699 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 0s 38ms/step - loss: 1.8857 - accuracy: 0.6158 - val_loss: 2.4094 - val_accuracy: 0.0000e+00
Epoch 3/5
5/5 [=

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
34/34 [==============================] - 2s 57ms/step - loss: 0.3709 - accuracy: 0.8637 - val_loss: 0.3561 - val_accuracy: 0.9816
Epoch 2/5
34/34 [==============================] - 2s 52ms/step - loss: 0.0888 - accuracy: 0.9835 - val_loss: 0.1665 - val_accuracy: 0.9823
Epoch 3/5
34/34 [==============================] - 2s 53ms/step - loss: 0.0793 - accuracy: 0.9838 - val_loss: 0.1296 - val_accuracy: 0.9830
Epoch 4/5
34/34 [==============================] - 2s 55ms/step - loss: 0.0751 - accuracy: 0.9839 - val_loss: 0.1103 - val_accuracy: 0.9833
Epoch 5/5
34/34 [==============================] - 2s 53ms/step - loss: 0.0724 - accuracy: 0.9839 - val_loss: 0.0949 - val_accuracy: 0.9836
Epoch 1/5
5/5 [==============================] - 0s 69ms/step - loss: 2.2522 - accuracy: 0.6158 - val_loss: 4.6121 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 0s 48ms/step - loss: 1.9397 - accuracy: 0.6158 - val_loss: 3.9082 - val_accuracy: 0.0000e+00
Epoch 3/5
5/5 [=

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0024


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.2169039249420166, 0.0], [1.0300925970077515, 0.0], [0.9772835969924927, 0.0]]
Accuracy for iterations:  [[0.8458676934242249, 0.023075968027114868], [0.8422871828079224, 0.00669947499409318], [0.7962055802345276, 0.002357222605496645]]
F1 for iterations:  [[0.8277851954926706, 0.045110958405756095], [0.8328297901688606, 0.013309781045885882], [0.8020408108831394, 0.0047033583629012285]]
Precision for iterations:  [[0.826770702008479, 1.0], [0.932864971064535, 1.0], [0.9321506112898233, 1.0]]
Recall for iterations:  [[0.8458676917353835, 0.023075968735784293], [0.8422871845743691, 0.006699474794259956], [0.7962055924111848, 0.0023572226127951696]]


#### DATASET 3E

In [42]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3E-Part3.csv')

In [43]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [44]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id004.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [45]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [46]:
global_model = build_model((24,1,1))

In [47]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [48]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [49]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id004.hdf5')

Epoch 1/5
126/126 [==============================] - 6s 44ms/step - loss: 0.4373 - accuracy: 0.8791 - val_loss: 0.9267 - val_accuracy: 0.9998
Epoch 2/5
126/126 [==============================] - 5s 42ms/step - loss: 0.0120 - accuracy: 1.0000 - val_loss: 0.4263 - val_accuracy: 0.9998
Epoch 3/5
126/126 [==============================] - 5s 43ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1134 - val_accuracy: 0.9999
Epoch 4/5
126/126 [==============================] - 5s 44ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0156 - val_accuracy: 1.0000
Epoch 5/5
126/126 [==============================] - 5s 43ms/step - loss: 6.4622e-04 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 1/5
17/17 [==============================] - 1s 64ms/step - loss: 4.8017 - accuracy: 0.0019 - val_loss: 1.9465 - val_accuracy: 0.0625
Epoch 2/5
17/17 [==============================] - 1s 53ms/step - loss: 2.0721 - accuracy: 0.1507 - val_loss: 0.7646 - val_accuracy: 0.8245
Epoch 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0863


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
126/126 [==============================] - 7s 56ms/step - loss: 0.0121 - accuracy: 0.9989 - val_loss: 4.3860e-04 - val_accuracy: 1.0000
Epoch 2/5
126/126 [==============================] - 7s 55ms/step - loss: 3.7278e-04 - accuracy: 1.0000 - val_loss: 1.1796e-04 - val_accuracy: 1.0000
Epoch 3/5
126/126 [==============================] - 7s 55ms/step - loss: 1.2937e-04 - accuracy: 1.0000 - val_loss: 6.1705e-05 - val_accuracy: 1.0000
Epoch 4/5
126/126 [==============================] - 7s 55ms/step - loss: 6.4674e-05 - accuracy: 1.0000 - val_loss: 3.7219e-05 - val_accuracy: 1.0000
Epoch 5/5
126/126 [==============================] - 7s 54ms/step - loss: 3.8021e-05 - accuracy: 1.0000 - val_loss: 2.4395e-05 - val_accuracy: 1.0000
Epoch 1/5
17/17 [==============================] - 1s 54ms/step - loss: 0.6713 - accuracy: 0.7418 - val_loss: 0.1951 - val_accuracy: 0.9958
Epoch 2/5
17/17 [==============================] - 1s 44ms/step - loss: 0.1412 - accuracy: 0.9980 - val_loss: 0.07

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
126/126 [==============================] - 7s 53ms/step - loss: 4.2822e-04 - accuracy: 0.9999 - val_loss: 1.8471e-05 - val_accuracy: 1.0000
Epoch 2/5
126/126 [==============================] - 7s 52ms/step - loss: 1.9617e-05 - accuracy: 1.0000 - val_loss: 9.0101e-06 - val_accuracy: 1.0000
Epoch 3/5
126/126 [==============================] - 7s 54ms/step - loss: 9.5984e-06 - accuracy: 1.0000 - val_loss: 5.5314e-06 - val_accuracy: 1.0000
Epoch 4/5
126/126 [==============================] - 7s 54ms/step - loss: 5.9301e-06 - accuracy: 1.0000 - val_loss: 3.8234e-06 - val_accuracy: 1.0000
Epoch 5/5
126/126 [==============================] - 7s 52ms/step - loss: 4.0489e-06 - accuracy: 1.0000 - val_loss: 2.9155e-06 - val_accuracy: 1.0000
Epoch 1/5
17/17 [==============================] - 1s 50ms/step - loss: 0.1866 - accuracy: 0.9894 - val_loss: 0.0461 - val_accuracy: 1.0000
Epoch 2/5
17/17 [==============================] - 1s 41ms/step - loss: 0.0333 - accuracy: 1.0000 - val_loss: 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.1091


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.20791494846344, 0.0], [0.8782169222831726, 0.0], [0.9215601086616516, 0.0]]
Accuracy for iterations:  [[0.34155869483947754, 0.08630742877721786], [0.8168051242828369, 0.1211281567811966], [0.8319796919822693, 0.10905256122350693]]
F1 for iterations:  [[0.4290826996459166, 0.1589005634231765], [0.748544589457651, 0.21608262633714498], [0.7558243931251452, 0.19665896039973155]]
Precision for iterations:  [[0.6608090572287052, 1.0], [0.7291052690660783, 1.0], [0.7207542089862083, 1.0]]
Recall for iterations:  [[0.34155868311736626, 0.0863074314544477], [0.8168051336102672, 0.12112815847152723], [0.8319796639593279, 0.10905256192878707]]


### Experimenting with epochs and iterations 

Use of dataset Cat3B, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [51]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')

In [52]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [53]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id010.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [54]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [55]:
global_model = build_model((24,1,1))

In [56]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [57]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [58]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id010.hdf5')

Epoch 1/10
15/15 [==============================] - 1s 67ms/step - loss: 1.0098 - accuracy: 0.6693 - val_loss: 1.4186 - val_accuracy: 0.0088
Epoch 2/10
15/15 [==============================] - 1s 43ms/step - loss: 0.5051 - accuracy: 0.9604 - val_loss: 1.4550 - val_accuracy: 8.2463e-04
Epoch 3/10
15/15 [==============================] - 1s 38ms/step - loss: 0.3097 - accuracy: 0.9615 - val_loss: 1.4905 - val_accuracy: 2.7488e-04
Epoch 4/10
15/15 [==============================] - 1s 38ms/step - loss: 0.2393 - accuracy: 0.9614 - val_loss: 1.5210 - val_accuracy: 0.0000e+00
Epoch 5/10
15/15 [==============================] - 1s 39ms/step - loss: 0.2107 - accuracy: 0.9616 - val_loss: 1.5484 - val_accuracy: 0.0000e+00
Epoch 6/10
15/15 [==============================] - 1s 39ms/step - loss: 0.1973 - accuracy: 0.9616 - val_loss: 1.5743 - val_accuracy: 2.7488e-04
Epoch 7/10
15/15 [==============================] - 1s 38ms/step - loss: 0.1890 - accuracy: 0.9614 - val_loss: 1.6021 - val_accuracy: 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0218


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
15/15 [==============================] - 1s 58ms/step - loss: 0.1481 - accuracy: 0.9652 - val_loss: 4.4030 - val_accuracy: 0.0022
Epoch 2/10
15/15 [==============================] - 1s 43ms/step - loss: 0.1456 - accuracy: 0.9651 - val_loss: 4.6469 - val_accuracy: 0.0019
Epoch 3/10
15/15 [==============================] - 1s 46ms/step - loss: 0.1451 - accuracy: 0.9656 - val_loss: 4.8806 - val_accuracy: 0.0019
Epoch 4/10
15/15 [==============================] - 1s 50ms/step - loss: 0.1446 - accuracy: 0.9664 - val_loss: 5.1195 - val_accuracy: 0.0019
Epoch 5/10
15/15 [==============================] - 1s 44ms/step - loss: 0.1437 - accuracy: 0.9665 - val_loss: 5.3554 - val_accuracy: 0.0019
Epoch 6/10
15/15 [==============================] - 1s 39ms/step - loss: 0.1430 - accuracy: 0.9665 - val_loss: 5.6345 - val_accuracy: 0.0019
Epoch 7/10
15/15 [==============================] - 1s 38ms/step - loss: 0.1426 - accuracy: 0.9666 - val_loss: 5.8514 - val_accuracy: 0.0019
Epoch 8/10
15

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 84/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0264  

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0305


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [59]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.0246622562408447, 0.0], [1.4416749477386475, 0.0], [1.683904767036438, 0.0], [1.8431316614151, 0.0], [1.9341689348220825, 0.0], [2.0031845569610596, 0.0], [2.056488275527954, 0.0], [2.0527660846710205, 0.0], [2.110060453414917, 0.0], [2.130955934524536, 0.0]]
Accuracy for iterations:  [[0.8624372482299805, 0.02183532528579235], [0.35324570536613464, 0.015714816749095917], [0.10781821608543396, 0.017327653244137764], [0.10784921795129776, 0.012116950936615467], [0.10784921795129776, 0.012778627686202526], [0.10789571702480316, 0.016376493498682976], [0.10801971703767776, 0.018692361190915108], [0.10801971703767776, 0.019643520936369896], [0.10794221609830856, 0.026839254423975945], [0.10792671889066696, 0.03051983006298542]]
F1 for iterations:  [[0.8570724759480345, 0.04273746408191347], [0.40902853126000055, 0.030943365498147468], [0.023505237229824536, 0.034065040650406504], [0.025493754426870276, 0.023943777069543187], [0.02583190357047961, 0.025234789710085

#### #Epochs = 5, #Iterations = 5

In [60]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat3B-Part3.csv')

In [61]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [62]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id011.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [63]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [64]:
global_model = build_model((24,1,1))

In [65]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)


In [66]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [67]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id011.hdf5')

Epoch 1/5
15/15 [==============================] - 1s 55ms/step - loss: 1.3763 - accuracy: 0.3948 - val_loss: 1.3997 - val_accuracy: 0.0489
Epoch 2/5
15/15 [==============================] - 1s 42ms/step - loss: 0.6559 - accuracy: 0.9176 - val_loss: 1.4343 - val_accuracy: 0.0027
Epoch 3/5
15/15 [==============================] - 1s 42ms/step - loss: 0.3593 - accuracy: 0.9532 - val_loss: 1.4677 - val_accuracy: 0.0019
Epoch 4/5
15/15 [==============================] - 1s 41ms/step - loss: 0.2545 - accuracy: 0.9544 - val_loss: 1.4967 - val_accuracy: 0.0014
Epoch 5/5
15/15 [==============================] - 1s 41ms/step - loss: 0.2151 - accuracy: 0.9564 - val_loss: 1.5228 - val_accuracy: 0.0011
Epoch 1/5
15/15 [==============================] - 1s 58ms/step - loss: 0.1838 - accuracy: 0.9619 - val_loss: 1.5861 - val_accuracy: 0.0019
Epoch 2/5
15/15 [==============================] - 1s 45ms/step - loss: 0.1692 - accuracy: 0.9627 - val_loss: 1.6404 - val_accuracy: 0.0022
Epoch 3/5
15/15 [===

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 76/756 [==>...........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0181

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0085: 0s - loss: 0.0


...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


 69/756 [=>............................] - ETA: 1s - loss: 0.0000e+00 - accuracy: 9.0580e-04

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


756/756 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0043


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [68]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.0814100503921509, 0.0], [0.9628205895423889, 0.0], [1.1502130031585693, 0.0], [1.2967842817306519, 0.0], [1.3596251010894775, 0.0]]
Accuracy for iterations:  [[0.8679397106170654, 0.008477730676531792], [0.8637547492980957, 0.008519085124135017], [0.8749767541885376, 0.00334973749704659], [0.6245737671852112, 0.0034738017711788416], [0.6486142873764038, 0.004300897475332022]]
F1 for iterations:  [[0.8503178269932039, 0.016812925449028134], [0.8459715613697972, 0.016894246934842337], [0.8587310410291272, 0.0066771082351001565], [0.6754417817454541, 0.006923552441788584], [0.6922036771336822, 0.008564957792876261]]
Precision for iterations:  [[0.8395150629795601, 1.0], [0.8357970173391828, 1.0], [0.8536917089703516, 1.0], [0.8037978674768768, 1.0], [0.787846126265895, 1.0]]
Recall for iterations:  [[0.8679397358794717, 0.008477730449526488], [0.863754727509455, 0.008519085232207105], [0.8749767499534999, 0.003349737397129978], [0.6245737491474983, 0.003473801745